In [1]:
import os
import pickle
import pandas as pd
os.chdir('C:/Users/user/Desktop/박준영/공부/nlp_study/simcse/classification-of-equity-report-based-on-contrastive-learning/unsimcse_using_pytorch/')
from util import pickle_to_dict, Args, save_new_data_to_pickle, excel_to_pandas


c:\Users\user\Anaconda3\envs\base_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 피클 파일
path = "C:/Users/user/Desktop/박준영/공부/nlp_study/simcse/data/pickle_180_days_compare_kospi/yes_data/"
data_list = os.listdir(path)
len(data_list)

data_dict = {}


for data_name in data_list:
    with open(path + data_name, 'rb') as f:
        data_dict[data_name] = pickle.load(f)

1. 데이터 키를 긁어온다.
2. 데이터 키에서 수익률 파트를 float형태로 바꾼다.
3. float형태로 바꾼 데이터셋에서 n 일로 잘라서 상관관계 비교
4. 상관관계 비교해서 가진행의 이름을 찾는다.
5. anchor, negative, positive 데이터를 붙인다.

In [4]:
data_dict['20220426.pkl'].drop(index = [781223], inplace=True)

In [5]:
def make_supervised_data(data_dict, days):
    """
    supervised dataset을 만드는 코드입니다.

    input:
        data_dict(dict) : 일자별 수익률 데이터
        days(int) : 수익률 기준일

    return:
        supervised data(dict) : {file_key : anchor, positive, negative 데이터셋}    
    """
    calculate_dict = {}
    uncalculate_dict = {}

    for file_key in data_dict.keys():
        # datadict의 수익률을 float형태로 바꾸는 코드입니다.
        data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
        # 행들의 상관관계를 구하는 코드입니다.
        # Nan 값이 있으면

        if data_dict[file_key].loc[:, 0:days-1].isnull().sum().sum() >0:
            print('not_done', file_key)
            uncalculate_dict[file_key] = data_dict[file_key]

        
        else:
            correlation_data = data_dict[file_key].loc[:, 0:days-1].transpose().corr() 
            for row in correlation_data.index:
                # 해당 행에 대한 상관관계 값
                current_row_correlations = correlation_data.loc[row].drop(row)
                min_corr_row = current_row_correlations.idxmin()
                max_corr_row = current_row_correlations.idxmax()
                data_dict[file_key].loc[row, ['anchor', 'negative', 'positive']] =data_dict[file_key].loc[row]['텍스트'], data_dict[file_key].loc[min_corr_row]['텍스트'], data_dict[file_key].loc[max_corr_row]['텍스트']
            
            calculate_dict[file_key] = data_dict[file_key].loc[:, ['anchor', 'negative', 'positive']]
            print('done', file_key)
    return uncalculate_dict, calculate_dict
        




        


In [7]:
imp_data_dict= data_dict.copy()

In [8]:
uncalculate_dict, calculate_dict = make_supervised_data(imp_data_dict, 5)

done 20210408.pkl
done 20210409.pkl
done 20210412.pkl


C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will a

done 20210413.pkl
done 20210414.pkl
done 20210415.pkl


C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will a

done 20210416.pkl
done 20210419.pkl
done 20210420.pkl
done 20210421.pkl


...

C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will a

done 20220509.pkl
done 20220510.pkl
done 20220511.pkl


C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will a

done 20220512.pkl
not_done 20220513.pkl
not_done 20220516.pkl
not_done 20220517.pkl
not_done 20220518.pkl


C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict[file_key].loc[:, 0:days-1] = data_dict[file_key].loc[:, 0:days-1].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_27636\1462086335.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will a

In [23]:
save_new_data_to_pickle('C:\\Users\\user\\Desktop\\박준영\\공부\\nlp_study\\simcse\\data\\supervised\\5days_sup_data\\', uncalculate_dict)
save_new_data_to_pickle('C:\\Users\\user\\Desktop\\박준영\\공부\\nlp_study\\simcse\\data\\supervised\\5days_sup_data\\', calculate_dict)

complete make dir
20220513.pkl save complete
20220516.pkl save complete
20220517.pkl save complete
20220518.pkl save complete
complete make dir
20171115.pkl save complete
20171116.pkl save complete
20171117.pkl save complete
20171120.pkl save complete
20171121.pkl save complete
20171122.pkl save complete
20171123.pkl save complete
20171124.pkl save complete
20171127.pkl save complete
20171128.pkl save complete
20171129.pkl save complete
20171130.pkl save complete
20171201.pkl save complete
20171204.pkl save complete
20171205.pkl save complete
20171206.pkl save complete
20171207.pkl save complete
20171208.pkl save complete
20171211.pkl save complete
20171212.pkl save complete
20171213.pkl save complete
20171214.pkl save complete
20171215.pkl save complete
20171218.pkl save complete
20171219.pkl save complete
20171220.pkl save complete
20171221.pkl save complete
20171222.pkl save complete
20171226.pkl save complete
20171227.pkl save complete
20171228.pkl save complete
20180102.pkl save c

In [19]:
calculate_dict.keys()

dict_keys(['20171115.pkl', '20171116.pkl', '20171117.pkl', '20171120.pkl', '20171121.pkl', '20171122.pkl', '20171123.pkl', '20171124.pkl', '20171127.pkl', '20171128.pkl', '20171129.pkl', '20171130.pkl', '20171201.pkl', '20171204.pkl', '20171205.pkl', '20171206.pkl', '20171207.pkl', '20171208.pkl', '20171211.pkl', '20171212.pkl', '20171213.pkl', '20171214.pkl', '20171215.pkl', '20171218.pkl', '20171219.pkl', '20171220.pkl', '20171221.pkl', '20171222.pkl', '20171226.pkl', '20171227.pkl', '20171228.pkl', '20180102.pkl', '20180103.pkl', '20180104.pkl', '20180105.pkl', '20180108.pkl', '20180109.pkl', '20180110.pkl', '20180111.pkl', '20180112.pkl', '20180115.pkl', '20180116.pkl', '20180117.pkl', '20180118.pkl', '20180119.pkl', '20180122.pkl', '20180123.pkl', '20180124.pkl', '20180125.pkl', '20180126.pkl', '20180129.pkl', '20180130.pkl', '20180131.pkl', '20180201.pkl', '20180202.pkl', '20180205.pkl', '20180206.pkl', '20180207.pkl', '20180208.pkl', '20180209.pkl', '20180212.pkl', '20180213.pkl

In [20]:
calculate_dict['20171115.pkl']['positive'].nunique()

144

In [21]:
calculate_dict['20171115.pkl']['negative'].nunique()

75

In [22]:
calculate_dict['20171115.pkl']

,anchor,negative,positive
주가번호,,,
383646,기업은행: 2017년(E) 지배주주순이익 +30.2%yoy인 1.51조원을 예상한...,"KT (030200) NDR takeaways - 본업은 양호, 규제가 변수 ● 3...",한국항공우주 (047810) 어닝쇼크지만 더 나빠질 건 없다 3Q17 Review:...
383617,"막연한 우려 대비 견조한 펀더멘탈3분기 지배순익 2,152억원(YoY +115.1%...","고려아연 (010130.KS) 아연 가격이 10년래 최고치로 상승했지만, 고려아연 ...",실적 Review2017.11.15 What’s new : 제일제당의 삼성생명 지분...
383236,실적 Review2017.11.15 What’s new : 제일제당의 삼성생명 지분...,"고려아연 (010130.KS) 아연 가격이 10년래 최고치로 상승했지만, 고려아연 ...","막연한 우려 대비 견조한 펀더멘탈3분기 지배순익 2,152억원(YoY +115.1%..."
383453,이제는 내년 소비재 주가상승 기대감으로 매수 Nov 201715 ▶ Highligh...,"고려아연 (010130.KS) 아연 가격이 10년래 최고치로 상승했지만, 고려아연 ...","막연한 우려 대비 견조한 펀더멘탈3분기 지배순익 2,152억원(YoY +115.1%..."
383647,기업은행: 2017년(E) 지배주주순이익 +30.2%yoy인 1.51조원을 예상한...,진성티이씨 (036890.KQ)미국 법인도 본격 실적 성장 본사 및 미국 법인 매출...,"Results Comment BUY40,00030,700 - 목표주가는 2018년 ..."
...,...,...,...
383209,"Results Comment 104,00071,400 투자의견 BUY, 목표주가 1...",2017년 실적 개선을 보여주며 주가 상승에 동참. 2018년 현대제철의 특수강공장...,"3분기 실적 리뷰: 시장 기대치 상회 3분기 연결 매출 5,693원(YoY +224..."
383195,"3분기 실적 리뷰: 시장 기대치 상회 3분기 연결 매출 5,693원(YoY +224...",2017년 실적 개선을 보여주며 주가 상승에 동참. 2018년 현대제철의 특수강공장...,"Results Comment 104,00071,400 투자의견 BUY, 목표주가 1..."
383305,"- 국내 턴어라운드는 시장의 기대보다 가파른 상황이며, Acushnet와 USA도 ...",2017년 실적 개선을 보여주며 주가 상승에 동참. 2018년 현대제철의 특수강공장...,"Results Comment 104,00071,400 투자의견 BUY, 목표주가 1..."
